[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/witchapong/build-ai-based-applications/blob/main/llm/1_basic_llm_applications.ipynb)

# Basic LLMs Applications

In this notebook, we'll learn to build basic LLMs applications using [LangChain](https://python.langchain.com/docs/introduction/), a framework for simplifying development of LLM application.

The content of this notebook is mainly modified from content from [KBTG M.A.D.Bootcamp](https://kbtgkampus.tech/).

In [ ]:
!pip install -q langchain==0.2.2
!pip install -q langchain_community==0.2.3
!pip install -q langchain-openai==0.1.8

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.6/973.6 kB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 397.1/397.1 kB 30.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.8/311.8 kB 24.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 22.4 MB/s eta 0:00:00


## Generate OpenAI key

Generate key from: https://platform.openai.com/api-keys

In [ ]:
import os

# TODO: put your generated openai key here
os.environ["OPENAI_API_KEY"] = "YOUR-SECRET-API-KEY"

## Call OpenAI using Langchain Model
Here, we'll try sending text to ChatGPT via API through Langchain library. Instead of chating with ChatGPT through a message box on using its [web-interface](https://chatgpt.com/) like usual, we'll do it through coding instead!

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
from langchain_core.output_parsers import StrOutputParser

In [ ]:
# write langchain model
model = ChatOpenAI(model="gpt-3.5-turbo")

messages = [
    HumanMessage(content="Hello from SIIT Rangsit Campus!")
]

m = model.invoke(messages)
print(m)

content='Hello! How can I assist you today?' response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 16, 'total_tokens': 26, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-27be22d4-8344-40b3-aca1-6cb827986317-0' usage_metadata={'input_tokens': 16, 'output_tokens': 10, 'total_tokens': 26}


In [ ]:
print(m.content)

Hello! How can I assist you today?


In [ ]:
# write langchain parser
parser = StrOutputParser()

In [ ]:
parser.invoke(m)

'Hello! How can I assist you today?'

In [ ]:
# chain langchain model and parser
chain = model | parser

In [ ]:
chain.invoke(messages)

'Hello! How can I assist you today?'

## Application-1: Improve Grammar and Stylize Writing using Langchain PromptTemplate
Here, we'll create a program that receives input English text or sentences and a language style that we want to augment the input text. Our program will then output grammar-corrected and stylized English text.

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

In [ ]:
system_template = "You will be provided with statements, and your task is to convert them to standard English."

style_prompts = {
    "default": "",
    "academic": " Ensure that the language used is appropriate for an academic research publication.",
    "ielts": " Using fancy words. Ensure that the language used meets the standards required for an IELTS score of 8.0.",
    "informal": " Make it informal like talking to a friend.",
    "royal": " Using royal vocabularies as if speaking with members of a royal family."
}

prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", system_template + "{style}"),
        ("user", "{text}")
    ]
)

user_text = "Hello agan mi freind. Where did you just went lastnight?"
user_style = "royal"

prompt_template.invoke({"text" : user_text, "style": style_prompts[user_style]})

ChatPromptValue(messages=[SystemMessage(content='You will be provided with statements, and your task is to convert them to standard English. Using royal vocabularies as if speaking with members of a royal family.'), HumanMessage(content='Hello agan mi freind. Where did you just went lastnight?')])

In [ ]:
# chain everything together

chain = prompt_template | model | parser

In [ ]:
chain.invoke({"text": user_text, "style": style_prompts[user_style]})

'Greetings, my dear friend. May I inquire as to where you ventured last night?'

In [ ]:
def improve_grammar(text, style="default"):
    # TODO-7: write function

    style_text_prompt = style_prompts[style]
    return chain.invoke({"text": text, "style": style_text_prompt})

In [ ]:
# TODO-8: test your function
user_text = "Hello again mine frinnds!"
improve_grammar(user_text)

'Hello again, my friends!'

In [ ]:
improve_grammar(user_text, "ielts")

'Greetings once more, my esteemed companions!'

In [ ]:
improve_grammar(user_text, "informal")

"Hey friends, how's it going?"

In [ ]:
improve_grammar(user_text, "royal")

'Greetings once more, my friends!'

## Application-2: Chatbot
In this part, we'll build a history-aware chatbot.

In [ ]:
model = ChatOpenAI(model="gpt-3.5-turbo")

In [ ]:
messages = [
    HumanMessage(content="Hello my name is MICK!")
]

m = model.invoke(messages)
print(m.content)

Hello MICK! How can I assist you today?


In [ ]:
messages = [
    HumanMessage(content="What's my name?")
]

m = model.invoke(messages)
print(m.content)

I'm sorry, I do not have access to personal information about users.


In [ ]:
# include history
messages = [
    HumanMessage(content="Hello my name is MICK!"),
    AIMessage(content="Hello MICK! Nice to meet you! How can I assist you today?"),
    HumanMessage(content="What's my name?")
]

m = model.invoke(messages)
print(m.content)

Your name is MICK.


In [ ]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

In [ ]:
# wrap model object
with_message_history = RunnableWithMessageHistory(model, get_session_history)

In [ ]:
config_1 = {"configurable": {"session_id": "room-0001"}}

In [ ]:
response = with_message_history.invoke(
    [HumanMessage(content="Hi! I'm Mick!")],
    config=config_1,
)

print(response.content)

Hello Mick! How are you doing today?


In [ ]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config_1,
)

print(response.content)

Your name is Mick!


In [ ]:
print(store)

{'room-0001': InMemoryChatMessageHistory(messages=[HumanMessage(content="Hi! I'm Mick!"), AIMessage(content='Hello Mick! How are you doing today?', response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 13, 'total_tokens': 23, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-6ec70121-19c8-4a19-9b63-15c160aa486c-0', usage_metadata={'input_tokens': 13, 'output_tokens': 10, 'total_tokens': 23}), HumanMessage(content="What's my name?"), AIMessage(content='Your name is Mick!', response_metadata={'token_usage': {'completion_tokens': 6, 'prompt_tokens': 35, 'total_tokens': 41, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens'

In [ ]:
config_2 = {"configurable": {"session_id": "room-0002"}}

In [ ]:
# TODO-11: explore store
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config_2,
)

print(response.content)

I'm sorry, I do not have the ability to know your name unless you tell me.


In [ ]:
store

{'room-0001': InMemoryChatMessageHistory(messages=[HumanMessage(content="Hi! I'm Mick!"), AIMessage(content='Hello Mick! How are you doing today?', response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 13, 'total_tokens': 23, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-6ec70121-19c8-4a19-9b63-15c160aa486c-0', usage_metadata={'input_tokens': 13, 'output_tokens': 10, 'total_tokens': 23}), HumanMessage(content="What's my name?"), AIMessage(content='Your name is Mick!', response_metadata={'token_usage': {'completion_tokens': 6, 'prompt_tokens': 35, 'total_tokens': 41, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens'

### Personalize Chatbot
Now, let's personalize out chatbot by giving it a name.

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt_template = ChatPromptTemplate.from_messages(
    [
        # add persona
        (
            "system",
            "You are a helpful assistant named \"Orca\""
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt_template | model

In [ ]:
store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

with_message_history = RunnableWithMessageHistory(chain, get_session_history)

In [ ]:
config = {"configurable": {"session_id": "room-0001"}}

In [ ]:
# test your bot persona
response = with_message_history.invoke(
    [HumanMessage(content="Hi! I'm Mick!")],
    config=config,
)

print(response.content)

Hello Mick! How can I assist you today?


In [ ]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

print(response.content)

Your name is Mick. How can I assist you today, Mick?


In [ ]:
response = with_message_history.invoke(
    [HumanMessage(content="What's your name?")],
    config=config,
)

print(response.content)

My name is Orca. I'm here to help you with any questions or tasks you have. Just let me know how I can assist you today!
